In [1]:
import numpy as np
import scipy as sp
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from lanczos_bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [3]:
def get_a_priori_bound_Qz(f,gamma,endpts,k,w,lminl,lmaxl,lminr,lmaxr):
    """
    (1/2pi) \oint_{\Gamma} |f(z)| (Q_{w,z})^{k+1} |dz|
    """
    
    def F(t):
        z,dz = gamma(t)
                
        return (1/(2*np.pi)) * np.abs(f(z)) * Q_wz(w,z,lminl,lmaxr)**(2*k) * np.max([Q_z(z,lminl,lmaxl),Q_z(z,lminr,lmaxr)]) * np.abs(dz)
    
    integral = sp.integrate.quad(F,endpts[0],endpts[1],epsabs=0,limit=200) 
    
    return integral


In [4]:
def get_a_posteriori_bound_Qz(f,gamma,endpts,a_,b_,w,lminl,lmaxl,lminr,lmaxr):
    """
    (1/2pi) \oint_{\Gamma} |f(z)| |D_{k,z}| Q_{w,z} |dz|
    """
    
    theta = sp.linalg.eigvalsh_tridiagonal(a_,b_,tol=1e-30)

    def F(t):
        z,dz = gamma(t)
        
        return (1/(2*np.pi)) * np.abs(f(z)) * np.abs(np.prod((theta-w)/(theta-z)))**2 * np.max([Q_z(z,lminl,lmaxl),Q_z(z,lminr,lmaxr)]) * np.abs(dz)
    
    integral = sp.integrate.quad(F,endpts[0],endpts[1],epsabs=0,limit=200) 
    
    return integral

## Setup

In [5]:
lam = np.load('matrices/MNIST_cov_eigs.npy')
n = len(lam)

b = np.ones(n,dtype=np.longdouble)
b /= np.linalg.norm(b)

In [6]:
a = np.max(lam)*.15
w = a

# norm to measure function approximation and linear system errors
B = np.ones(n)

lminl = np.min(lam)
lmaxl = np.max(lam[lam<a])
lminr = np.min(lam[lam>a])
lmaxr = np.max(lam)
kappa_w = np.max(np.abs(lam-w)) / np.min(np.abs(lam-w))

In [7]:
np.count_nonzero(lam>a)

16

In [8]:
K = 50
reorth=True
Q,(a_,b_) = exact_lanczos(lam.astype(np.longdouble),b.astype(np.longdouble),K+1,reorth=reorth)

In [9]:
plot_tol = 1e-14

In [10]:
T = np.zeros((3,K+1))
T[0,1:] = b_
T[1] = a_-w
T[2,:-1] = b_

e1 = np.zeros(K+1)
e1[0] = 1

In [11]:
err_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = (1/(lam-w))*b - np.linalg.norm(b)*Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1])
        err_CG[k] = np.sqrt( e.T*B@e )
    except: pass
    
    if err_CG[k] < plot_tol:
        break

In [12]:
res_CG = np.full(K,np.nan)

for k in range(0,K):
    try:
        r = b - np.linalg.norm(b)*(lam-w)*(Q[:,:k+1]@sp.linalg.solve_banded((1,1),T[:,:k+1],e1[:k+1]))
        res_CG[k] = np.sqrt( r@r )
    except: pass
    
    if res_CG[k] < plot_tol:
        break

## $f(x) = \operatorname{step}(x-a)$, $\Gamma =$ double circles

In [13]:
def f(x):
    return (np.abs(np.real(x))>a)

fAb = f(lam)*b

In [14]:
err_lanczos = np.full(K,np.nan)
err_quad_form = np.full(K,np.nan)

for k in range(0,K):
    try:
        e = fAb - lanczos_FA(f,Q[:,:k+1],a_[:k+1],b_[:k],normb=np.linalg.norm(b)) # B-norm of b is one
        err_lanczos[k] = np.sqrt( e.T*B@e )
        err_quad_form[k] = np.abs(b@e)

    except: pass
    
    if err_lanczos[k] < plot_tol:
        break

In [15]:
a_priori_bound = np.full(K,np.inf)
a_posteriori_bound = np.full(K,np.inf)

def Gamma1(t):
    radius = 10*lmaxr-w

    z = w + 1e-10 + 1j*t
    dz = 1j#radius*1j*np.exp(1j*t)
    
    return z,dz

def Gamma2(t):
    radius = 10*(w-lminl)

    z = w-1e-10+1j*t#radius*(np.exp(1j*t)) + (w-radius)
    dz = 1j#radius*1j*np.exp(1j*t)
    
    return z,dz


for k in range(1,K+1):
    a_priori_bound[k-1] = 2*(get_a_priori_bound_Qz(f,Gamma1,[0,np.inf],k,w,lminl,lmaxl,lminr,lmaxr)[0]+\
                             get_a_priori_bound_Qz(f,Gamma2,[0,np.inf],k,w,lminl,lmaxl,lminr,lmaxr)[0])
    a_posteriori_bound[k-1] = 2*(get_a_posteriori_bound_Qz(f,Gamma1,[0,np.inf],a_[:k],b_[:k-1],w,lminl,lmaxl,lminr,lmaxr)[0]+\
                                 get_a_posteriori_bound_Qz(f,Gamma2,[0,np.inf],a_[:k],b_[:k-1],w,lminl,lmaxl,lminr,lmaxr)[0])

In [16]:
a_priori_bound_approx_spectrum = np.full(K,np.inf)
a_posteriori_bound_approx_spectrum = np.full(K,np.inf)

def Gamma1(t):
    radius = 10*lmaxr-w

    z = w + 1e-10 + 1j*t
    dz = 1j#radius*1j*np.exp(1j*t)
    
    return z,dz

def Gamma2(t):
    radius = 10*(w-lminl)

    z = w-1e-10+1j*t#radius*(np.exp(1j*t)) + (w-radius)
    dz = 1j#radius*1j*np.exp(1j*t)
    
    return z,dz

gam = np.min(np.abs((w-lminl,w-lmaxl)))/100

for k in range(1,K+1):
    a_priori_bound_approx_spectrum[k-1] = 2*(get_a_priori_bound_Qz(f,Gamma1,[0,np.inf],k,w,lminl/2,w-gam,w+gam,lmaxr*2)[0]+\
                                             get_a_priori_bound_Qz(f,Gamma2,[0,np.inf],k,w,lminl/2,w-gam,w+gam,lmaxr*2)[0])
    a_posteriori_bound_approx_spectrum[k-1] = 2*(get_a_posteriori_bound_Qz(f,Gamma1,[0,np.inf],a_[:k],b_[:k-1],w,lminl/2,w-gam,w+gam,lmaxr*2)[0]+\
                                                 get_a_posteriori_bound_Qz(f,Gamma2,[0,np.inf],a_[:k],b_[:k-1],w,lminl/2,w-gam,w+gam,lmaxr*2)[0])

In [17]:
plt.figure(figsize=(6,4))

plt.plot(err_lanczos**2,linestyle='-',linewidth=1,color='#073642')

plt.plot(err_quad_form,linestyle='-',linewidth=2,color='#073642')

#plt.plot(sub_mult_bound,linestyle=(0,(4,2,1,3)),linewidth=1,color='#2aa198')
#plt.plot(triangle_bound,linestyle=(0,(7,5)),linewidth=1,color='#859900')
#plt.plot(full_bound,linestyle='-',linewidth=1,color='#b58900')

plt.plot(a_priori_bound*res_CG**2,linestyle='-.',linewidth=2,color='#6c71c4')
plt.plot(a_posteriori_bound*res_CG**2,linestyle='--',linewidth=2,color='#d33682')

plt.plot(a_priori_bound_approx_spectrum*res_CG**2,linestyle='-',linewidth=.5,color='#6c71c4')
plt.plot(a_posteriori_bound_approx_spectrum*res_CG**2,linestyle='-',linewidth=.5,color='#d33682')

#plt.plot((lmax-w)/w*err_CG)

#plt.plot(a_priori_bound*res_CG/lmin,linestyle='-.',linewidth=1,color='#6c71c4')
#plt.plot(a_posteriori_bound*res_CG/lmin,linestyle='--',linewidth=1,color='#d33682')


plt.yscale('log')

legend_elements = [ Line2D([0],[0],linestyle='-',linewidth=2,color='#073642',label=r'$| \mathbf{b}^{\footnotesize\textsf{H}} f(\mathbf{A}) \mathbf{b} - \mathbf{b}^{\footnotesize\textsf{H}} \textsf{lan}_k(f) |$'),

                   Line2D([0],[0],linestyle='--',linewidth=2,color='#d33682',label='$(\\textrm{Q1}) \: \| \\textsf{res}_k(w) \|_2^2$, $S=\\mathcal{I}_w(\mathbf{A})$'),
                   Line2D([0],[0],linestyle='-.',linewidth=2,color='#6c71c4',label='$(\\textrm{Q2}) \: \| \\textsf{res}_k(w) \|_2^2$, $S=\\mathcal{I}_w(\mathbf{A})$'),
                   
                   Line2D([0],[0],linestyle='-',linewidth=1,color='#073642',label='$\| f(\mathbf{{A}})\mathbf{{b}} - \\textsf{lan}_k(f) \|_2^2$'),

                   Line2D([0],[0],linestyle='-',linewidth=.5,color='#d33682',label='$S=\\tilde\\mathcal{I}_w(\mathbf{A})$'),
                   Line2D([0],[0],linestyle='-',linewidth=.5,color='#6c71c4',label='$S=\\tilde\\mathcal{I}_w(\mathbf{A})$'),

                   
#                   Line2D([0],[0],linestyle='-',linewidth=.5,color='#073642',label='$\\exp(-4k/\\kappa(\\mathbf{A}-w\\mathbf{I}))$')
                  ]

#plt.legend(handles=legend_elements,loc='lower left',ncol=2)

#for c in range(-15,2,2):
#    plt.plot(10**(c)*(( kappa_w - 1 )/( kappa_w + 1 ))**(2*np.arange(K)),linestyle=':',linewidth=.5,color='#073642')

plt.ylim(1e-16,1e1)

plt.grid(True,linestyle=':',linewidth=.5)

#plt.title('quadratic form error bounds: $f(x) = \\mathrm{step}(x-a)$,')
plt.xlabel('Lanczos iteration $k$')

plt.savefig('imgs/step_quadform.pdf',bbox_inches='tight')

RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 432x288 with 1 Axes>